### Gather articles from wikipedia to compare the similarity between them.

In [77]:
# Call the libraries
import concurrent.futures
import re
import nltk
from nltk.corpus import stopwords
from nltk import tokenize
from nltk.stem import SnowballStemmer
import requests

In [78]:
def get_wikipedia_article(wiki_article, timeout=10): # This function gathers the websites from wiki
    '''
    Method that gets a Wikipedia page. 
    '''
    wiki_page_url = 'https://en.wikipedia.org/wiki/'+wiki_article
    response = requests.get(url=wiki_page_url, timeout=timeout)
    article = response.text
    return article


def get_wikipedia_articles(wiki_articles):
    with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
        futures = []
        for article in wiki_articles:
            futures.append(executor.submit(get_wikipedia_article,
                                           wiki_article=article)
                          )
        articles = []
        for future in concurrent.futures.as_completed(futures):
            articles.append(future.result())
    return articles


# Let's define some functions to clean HTML code, punctuation marks and symbols from the original articles
def clean_html(raw_html):
    cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    cleantext = re.sub(cleanr, ' ', raw_html)
    return cleantext

def clean_punctuation_marks(text):
    return re.sub(r'[^\w\s]',' ', text)

def clean_return_and_tabs(text):
    return re.sub(r'[\n, \t]',' ', text)

def clean_article(data):
    cdata = clean_html(data)
    cdata = clean_punctuation_marks(cdata)
    cdata = clean_return_and_tabs(cdata)
    return cdata

<h2> The wiki articles

In [79]:
wiki_articles = [
    'lion',
    'beetle',
    'tiger',
]
articles = get_wikipedia_articles(wiki_articles)

#### Now that we have the articles, let's do some basic cleaning and NLP on them:

Basic cleaning -> tokenise -> remove stopwords -> stem -> remove numbers -> remove specific stopwords

In [80]:
tokeniser = tokenize.NLTKWordTokenizer()
stemmer = SnowballStemmer('english')

In [81]:
# First we make it all lowercase and do a basic cleaning
articles = map(lambda a: a.lower(), articles)
articles = map(clean_article, articles)
# Then we tokenise the articles, extracing the words from them
articles = map(tokeniser.tokenize, articles)

# We will also remove the English stopwords 

stopwords = set(['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 
                'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 
                'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 
                'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 
                'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 
                'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 
                'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 
                'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 
                'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 
                'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 
                'doing', 'it', 'how', 'further', 'was', 'here', 'than'])

def remove_stopwords(document, stopwords):
    output_doc = []
    for word in document:
        if word not in stopwords:
            output_doc.append(word)
    return output_doc

articles = map(lambda a: remove_stopwords(a, stopwords), articles)

# And then we stem those words to remove plurals and other variations
articles = [map(stemmer.stem, article) for article in articles]


def remove_numbers(document):
    numbers = set(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])
    output_doc = []
    for word in document:
        if not any(num in word for num in numbers):
            output_doc.append(word)
    return output_doc

articles = map(remove_numbers, articles)

def remove_wiki_and_wg_words(document):
    output_doc = []
    for word in document:
        if not word.startswith('wg') and not word.startswith('wiki'):
            output_doc.append(word)
    return output_doc

articles = map(remove_wiki_and_wg_words, articles)
articles = list(articles)

<h2>TF-IDF algorithm

Using this algorithm we can see the following:
* TF: How relevant a term is inside the article. TF = num. times term appears in article/num. terms in document
* IDF: Find the relevance of a term across all the documents. IDF = Log(num. documents in corpus/ num. documents where the term is in)

Term relevance score = TF * IDF

<h3> Term Frequency

In [85]:
def term_frequency(articles, word):
    N = len(articles)
    occurance = len([token for token in articles if token == word])
    
    return occurance / N

<h3>Inverse Document Frequency (IDF)

In [86]:
import numpy as np

In [87]:
def inverse_document_frequency(word, corpus):
    count_of_documents = len(corpus) + 1
    count_of_documents_with_word = sum([1 for doc in corpus if word in doc]) + 1
    idf = np.log10(count_of_documents/count_of_documents_with_word) + 1
    return idf

<h3> Find the TF-IDF

In [88]:
def calculate_tf_idf(articles):
    sorted_lists = []
    
    for article in articles:
        tf_idf_dict = {}
        for word in article:
            tf = term_frequency(article, word)
            idf = inverse_document_frequency(word, article)
            tf_idf_dict[word] = tf * idf
        
        sorted_words = sorted(tf_idf_dict, key=tf_idf_dict.get, reverse=True)[:20]
        sorted_lists.append(sorted_words)
    
    return sorted_lists

# Example usage:
articles = [articles[0], articles[1], articles[2]]  # Assuming articles is a list containing 3 lists
shorted = calculate_tf_idf(articles)
for sorted_words in shorted:
    print(sorted_words)

['tiger', 'mw', 'parser', 'output', 'n', 'doi', 'j', 'p', 'vector', 'toc', 'tigri', 'm', 'panthera', 'popul', 'conserv', 'b', 'rang', 'cat', 'speci', 'hlist']
['lion', 'mw', 'parser', 'output', 'toc', 'vector', 'p', 'doi', 'j', 'panthera', 'nation', 'africa', 'male', 'leo', 'list', 'm', 'african', 'hlist', 'cat', 'c']
['beetl', 'mw', 'parser', 'output', 'toc', 'vector', 'speci', 'insect', 'doi', 'coleoptera', 'list', 'level', 'class', 'edit', 'item', 'use', 'larva', 'li', 'hlist', 'id']


<h2> Jaccard Similarity

In [90]:
def jaccard(a1, a2):
    set1 = set(a1)
    set2 = set(a2)
    union_size = len(set1.union(set2))
    intersection_size = len(set1.intersection(set2))
    return round(intersection_size/union_size, 4)

for i in range(len(shorted)-1):
    for j in range(i+1, len(shorted)):
        print('Similarity between', shorted[i][0], 'and', shorted[j][0], ':',jaccard(shorted[i], shorted[j]))

Similarity between tiger and lion : 0.4286
Similarity between tiger and beetl : 0.25
Similarity between lion and beetl : 0.25


In [91]:
import binascii  # To transform (hash) each shingle to a 32-bit integer 
import random

In [92]:
def shingle_article(shorted, shingle_size=3):
    hashed_shingles = [] # Shingles are those 3 words transformed in an integer using CRC32 algorithm
    for i in range(len(shorted)-(shingle_size-1)):
        # We put groups (shingles) of 3 words together, separated by a space
        # We could change the number of words per shingle, changing shingle_size
        shingle = ' '.join([shorted[i+j] for j in range(shingle_size)])
        crc_hash = binascii.crc32(bytes(shingle.encode('UTF-8')))
        hashed_shingles.append(crc_hash)
    return hashed_shingles

SHINGLE_SIZE = 1

# We "save" the first word of the article in order to be able to idenfity it later when we hash it all 
shingled_articles = map(lambda a: (a[0], shingle_article(a, shingle_size=SHINGLE_SIZE)), shorted)
shingled_articles = list(shingled_articles)

In [93]:
maxCoefficient = 2**32-1  # Because we are transforming shingles into 32-bit integers, we know their maximum
big_prime = 4294967311  # And we found their next primer in internet

N = 20 # Using 20 hashing functions

# We will use N hashing functions, so we need to randomly take N As and N Bs for our N hashing functions
# Our hashing functions will be of the form: hashed_value = A * value + B % big_prime
a_coeffs = [random.randint(0, maxCoefficient) for i in range(N)]
b_coeffs = [random.randint(0, maxCoefficient) for i in range(N)]

In [94]:
def get_hashed_value(shingle, a_coeff, b_coeff, big_prime):
    return ((shingle * a_coeff) + b_coeff) % big_prime

# For each article, and each of the N hash functions, we calculate the hashes of all the shingles
# and get the minimum of those hashes. This will create a signature of size N for each document. 
# Finally, to estimate their Jaccard similarity, we just only compare the signatures
article_signatures = {}
article_names = []
for article_name, shingled_article in shingled_articles:
    article_signature = []
    for hashing_function_index in range(N):
        a_coeff = a_coeffs[hashing_function_index]
        b_coeff = b_coeffs[hashing_function_index]
        hashes = map(lambda shingles: get_hashed_value(shingles, a_coeff, b_coeff, big_prime), shingled_article)
        article_signature.append(min(list(hashes)))
    article_signatures[article_name] = article_signature
    article_names.append(article_name)
    
for s in article_signatures:
    print('Signature for', s,': ',article_signatures[s])

Signature for tiger :  [38153704, 120609307, 83707228, 106787783, 275718947, 602122745, 152872557, 34995198, 584691928, 135974804, 62540410, 383057044, 608270832, 178204528, 461075123, 343208120, 197815915, 122461160, 435865575, 286825418]
Signature for lion :  [750223927, 46802055, 508356304, 40027168, 55406979, 602122745, 160166190, 34995198, 178019522, 135974804, 62540410, 383057044, 313620813, 338554930, 241323024, 56378505, 197815915, 83689526, 77145839, 535766859]
Signature for beetl :  [438825287, 763506739, 508356304, 118347302, 487100142, 83334534, 63481854, 361993534, 99872824, 135974804, 62540410, 441775863, 99724279, 19631363, 145730619, 268221364, 141678703, 122461160, 187706360, 60918832]


In [95]:
print('Using', N, 'hashing functions, and shingles of size',SHINGLE_SIZE, ', MinHas produces the following results:')
for i in range(len(article_names)-1):
    for j in range(i+1, len(article_names)):
        print('Similarity between', article_names[i], 'and', article_names[j], ':',
              jaccard(article_signatures[article_names[i]], article_signatures[article_names[j]]))

Using 20 hashing functions, and shingles of size 1 , MinHas produces the following results:
Similarity between tiger and lion : 0.1765
Similarity between tiger and beetl : 0.0811
Similarity between lion and beetl : 0.0811
